In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [11]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        V: vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # TODO: Implement!
        threshold = 0
        for s in range(env.nS):
            state_value = 0
            for a in range(env.nA): # 이 경우는 state 별 가능한 action이 전체집합으로 모두 동일하기 때문에 env.nA로 해도 됨. 그렇지 않은 경우는 enumerate(policy[s]) 로 사용.
                state_action_value = 0
                for i in env.P[s][a]: # i: s에서 a를 했을 때 이동 가능한 다음 state 정보
                    """
                    solution에서는 s'에 대한 summation을 하지 않음.
                    gridworld env에서는 action을 했을 때 s'이 deterministic하게 결정되기 때문에 괜찮지만,
                    다른 env에서도 적용하려면 아래처럼 s'에 대한 summation을 진행한 다음에 계산하는게 맞지 않을까?
                    """
                    state_action_value += (i[0] * V[i[1]]) # i[0]: transition prob , i[1]: next state

                state_value += (policy[s][a] * (env.P[s][a][0][2] + discount_factor * state_action_value))

            threshold = max(threshold, np.abs(V[s]-state_value))
            """
            위에서 계산한 threshold는 특정 s에 대한 threshold 임.
            따라서 iteration 종료 조건에 사용하기 위해서는 모든 s에 대한 threshold 중 최대값을 구해야 함.
            """
            V[s] = state_value
        
        if threshold < theta:
            break
    return np.array(V)

In [12]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
v

array([  0.        , -13.99993529, -19.99990698, -21.99989761,
       -13.99993529, -17.9999206 , -19.99991379, -19.99991477,
       -19.99990698, -19.99991379, -17.99992725, -13.99994569,
       -21.99989761, -19.99991477, -13.99994569,   0.        ])

In [13]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)